In [1]:
# Veuillez renseigner les dates de backtest souhaitées : 
print("Veuillez Selectionner l'année de départ pour la fenêtre d'optimisation:")
année_début = input()
année_début = int(année_début)

print("Veuillez Selectionner le mois de départ pour la fenêtre d'optimisation:")
mois_début = input()
mois_début = int(mois_début)

print("Veuillez Selectionner le jour de départ pour la fenêtre d'optimisation:")
jour_début = input()
jour_début = int(jour_début)

print("Veuillez Selectionner l'année de fin pour la fenêtre d'optimisation:")
année_fin = input()
année_fin = int(année_fin)

print("Veuillez Selectionner le mois de fin pour la fenêtre d'optimisation:")
mois_fin = input()
mois_fin = int(mois_fin)

print("Veuillez Selectionner le jour de fin pour la fenêtre d'optimisation:")
jour_fin = input()
jour_fin = int(jour_fin)

# Veuillez renseigner la quantité de Short souhaitée : 
print("Veuillez Sélectionner la quantité de short que vous souhaitez avoir lors de l'optimisation (exemple : 0.4 = 40%)")
max_short_trehsold = input()
max_short_trehsold = int(max_short_trehsold)

# Veuillez renseigner l'optimisation que vous souhaitez : 
print("Veuillez Sélectionner l'optimisation que vous souhaitez : 1 pour un optimisation 93,63 pour un évantaille de MAR, 2 pour une optimisation 93,63 pour un seul MAR, 3 pour l'optimisation classique 63 & 93 avec les MAR habituels")
Choix_optimisation = input()
Choix_optimisation = int(Choix_optimisation)

# Importation des modules :
import openpyxl 
import pandas as pd
import numpy as np
import datetime
from datetime import date
from datetime import datetime
from openpyxl import load_workbook
from pandas.tseries.offsets import BDay
import matplotlib.pyplot as plt 
from matplotlib import style
plt.style.use('fivethirtyeight')
import os 
from os import walk
from sklearn.preprocessing import MinMaxScaler

color_liste = ["b","g","r","c","m","y","k","tab:pink","tab:gray","tab:orange","teal","lightgreen","peru","indianred","forestgreen","lemonchiffon","saddlebrown","peachpuff","lightblue","darkgrey","deeppink","violet"]

TODAY = date.today().strftime("%Y%m%d")
TODAY = '20221201'

directory_path = "Portefeuilles Directory/"

# Fonction qui permet d'obtenir les liste des portefeuilles à optimiser
# Input_Type : None
# Output_Type : String_List,String_List
def get_lists():
    
    listeFichiers = []
    
    for (fichiers) in walk(directory_path):
        listeFichiers.extend(fichiers)

    portefeuille_name_list = listeFichiers[2]

    portefeuille_list = [] 
    
    for value in listeFichiers[2]:
        portefeuille_list.append(directory_path + value)
        
    return portefeuille_list,portefeuille_name_list

# Declaration des listes : 
portefeuille_list,portefeuille_name_list = get_lists()

# Construction des filespath pour charger les portefeuilles : (théoriquement, leurs noms contient la date d'aujourd'hui)
builded_portefeuille_list = []
for portefeuille in portefeuille_name_list:
    builded_portefeuille_list.append('{}'.format(TODAY) + "_" + portefeuille)
    
folder_name = "Dossier_Portefeuilles_crées_convertis/" 

directory_list = []
for name in builded_portefeuille_list:
    directory_list.append(folder_name + name)

Liste_for_database_mngt = []
for value in portefeuille_name_list:
    Liste_for_database_mngt.append(value[:-5])

# Création des Dossiers dans lesquels vont être écris les résultats de l'optimisation : 
import shutil
# Suppression des dossiers existant : 
for k in range(len(Liste_for_database_mngt)):
    shutil.rmtree(Liste_for_database_mngt[k] + "/Weight matrices")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Statistiques")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Realized returns")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Profils allocation")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Performances VS Equity Curves")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Equity Curves")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Drawdowns")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Absolute returns")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Performances attribution")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Drawdowns csv")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Multiplots_Equity_Curves")
    shutil.rmtree(Liste_for_database_mngt[k] + "/Multiplots_Drawdowns")
# Création des Dossiers dans lesquels vont être écris les résultats de l'optimisation
# Suite au clean de l'arborescence, on re-créer ici l'ensemble des dossiers nécessaire au stockage des résultats : 
for k in range(len(Liste_for_database_mngt)):
    os.mkdir(Liste_for_database_mngt[k] + "/Weight matrices")
    os.mkdir(Liste_for_database_mngt[k] + "/Statistiques")
    os.mkdir(Liste_for_database_mngt[k] + "/Realized returns")
    os.mkdir(Liste_for_database_mngt[k] + "/Profils allocation")
    os.mkdir(Liste_for_database_mngt[k] + "/Performances VS Equity Curves")
    os.mkdir(Liste_for_database_mngt[k] + "/Equity Curves")
    os.mkdir(Liste_for_database_mngt[k] + "/Drawdowns")
    os.mkdir(Liste_for_database_mngt[k] + "/Absolute returns")
    os.mkdir(Liste_for_database_mngt[k] + "/Performances attribution")
    os.mkdir(Liste_for_database_mngt[k] + "/Drawdowns csv")
    os.mkdir(Liste_for_database_mngt[k] + "/Multiplots_Equity_Curves")
    os.mkdir(Liste_for_database_mngt[k] + "/Multiplots_Drawdowns") 
# Construction de la fonction pour obtenir les log returns :
# Input_Type : DataFrame
# Output_Type : DataFrame of Float
def log_ret(portefeuille):
    log_return = np.log(portefeuille/portefeuille.shift(1))
    return log_return  

# Fonction d'obtention des listes de portefeuilles : 
# Input_Type : DataFrame,int,int
# Output_Type : liste,DataFrame,DataFrame,DataFrame_List,DataFrame_List,DataFrame_List,DataFrame_List
def prepare_portefeuille(Portefeuille,window_backtest,window_lenght,backtest_starting_date,backtest_ending_date): 
    
    # Préparation des portefeuilles : 
    portefeuille = Portefeuille.copy()
    portefeuille.rename(columns = {"Unnamed: 0" : 'Date'},inplace = True)
    if "Date" in portefeuille.columns:
        
        portefeuille.set_index("Date",inplace = True)
    portefeuille = portefeuille[backtest_starting_date:backtest_ending_date]

    # Fill after et fill before des Nan Values :
    # Fill before :
    Portefeuille_before = portefeuille.ffill(axis = 'rows')
    # Fill after :
    Portefeuille_after = portefeuille.bfill(axis = 'rows')
    
    # Pour réaliser les backtest, nous avons besoin de données de la taille de window_backtest avant le premier backtest au minimum :
    Portefeuille_before_ = Portefeuille_before[window_backtest + 1:]
    # Découpage du portefeuille en une liste de portefeuille de taille window_lenght:
    Portefeuille_list = np.array_split(Portefeuille_before_, len(Portefeuille_before_)/window_lenght)
    
    # Création de l'index_list pour chacuns des portefeuilles :
    index_list = [Portefeuille_before.index[window_backtest]]
    for k in range (len(Portefeuille_list)-1):
        index_list.append(Portefeuille_list[k].index[len(Portefeuille_list[k].index)-1])
        
    # Création de la liste des portefeuilles bien coupés en bout, coupe à la fin :    
    Portefeuille_before_backtest_list = []
    for j in range (len(index_list)):
        Portefeuille_before_backtest_list.append(Portefeuille_before[:index_list[j]])
    Portefeuille_before_backtest_list.append(Portefeuille_before[index_list[len(index_list)-1]:])
    
    Portefeuille_before_backtest = []
    for k in range (len(Portefeuille_before_backtest_list)):
        Portefeuille_before_backtest.append(Portefeuille_before_backtest_list[k][-window_backtest:])
    
    # Création des portefeuilles de log_return pour calculer les poids :
    
    Log_return_Portefeuille_before_backtest_list = []
    for portefeuille in Portefeuille_before_backtest:
        Log_return_Portefeuille_before_backtest_list.append(log_ret(portefeuille))
        
    Log_return_Portefeuille_before_backtest_list = Log_return_Portefeuille_before_backtest_list[:-1]
    
    Log_return_Portefeuille_before_backtest_list.append(log_ret(Portefeuille_before[-window_backtest:]))
        
    return index_list,Portefeuille_before,Portefeuille_before_,Portefeuille_list,Portefeuille_before_backtest_list,Portefeuille_before_backtest,Log_return_Portefeuille_before_backtest_list

# Fonction de calcul des statistiques annualisée des portefeuilles : 
# Input_Type : Float_list,DataFrame
# Output_Type : Array (float,float,float)
def get_ret_vol_sr(weights,portefeuille):
        weights = np.array(weights) # Array
        ret = np.sum(portefeuille.mean() * weights) * 252 # Float
        vol = np.sqrt(np.dot(weights.T, np.dot(portefeuille.cov() * 252, weights))) # Float
        sr = ret/vol #Float
        return np.array([ret,vol,sr]) # Array
    
# Fonction qui permet de garder les poids égaux à 1 au niveau des matrices d'allocations : 
# Input_Type : Matrice_Row : float_List (== List)
# Output_Type : Matrice_Row (== List)
def sort_row(row):
    
    for data1,data2 in zip(np.abs(row),range(len(row))):
        if data1<= 0.001:
            row[data2] = 0 
            row[pd.DataFrame(row).idxmax()] = row[pd.DataFrame(row).idxmax()] + data1
            
    counter = np.abs(row).sum() - 1 
    row[pd.DataFrame(row).idxmax()] = row[pd.DataFrame(row).idxmax()] - counter
            
    return row

# Fonction d'optimisation des Portefeuilles : 
# Input_Type : DataFrame_List,Float,Int_List(== Array),Int,Float,Int
# Output_Type : DataFrame : Matrice_ret_vol_sharp,Array_List
def Make_Markowitz_Optimisation(Portefeuille_list,vol_max,Bounds_list,Portefeuille_lenght,max_short_trehsold,window_lenght):
    
    from scipy.optimize import minimize
    
    Weight_matrice = [] # Liste
    ret_vol_sr_matrice = np.zeros((len(Portefeuille_list)+1,3)) # Matrice(n x 3)
    
    i = 0
                                
    for portefeuille in Portefeuille_list:

        def neg_return(weights):
            return  get_ret_vol_sr(weights,portefeuille)[0] * - 1
        
        # Les contraintes sont gérées de la manière suivante : 
        # Zéro est la valeur maximale tolérée pour les inégalité et c'est la valeur à obtenir pour les égalitées.
        
        # Contrainte sur les Poids :
        
        def check_sum(weights):
            return np.sum(np.abs(weights)) - 1

        # Contrainte sur la Vol : 

        def check_vol(weights):
            RHS = vol_max - get_ret_vol_sr(weights, portefeuille)[1]
            return RHS
        
        # Contrainte sur les Short : 
        
        def check_max_short(weights):
            liste = [] # Liste
            for value in weights:
                if value<=0:
                    liste.append(value)
            return max_short_trehsold  - np.sum(np.abs(liste))
        
        # Construction des contraintes :
        
        cons = ({'type': 'eq', 'fun': check_sum}, {'type': 'ineq', 'fun': check_vol},{'type': 'ineq', 'fun': check_max_short})

        # Hypothèse initiale (Répartition égale) : 
    
        init_guess = [] 
        for k in range(Portefeuille_lenght):
            init_guess.append(1 / Portefeuille_lenght)
            
            # Optimisation "SLSQP":
            
        opt_results = minimize(neg_return, init_guess, bounds = Bounds_list, method='SLSQP', constraints=cons) 
        Weight_matrice.append(opt_results.x)
        ret_vol_sr_matrice[i,:] = get_ret_vol_sr(opt_results.x,portefeuille)
        
        i = i + 1
        
    ret_vol_sr_matrice = pd.DataFrame(ret_vol_sr_matrice)
    ret_vol_sr_matrice.columns = ["Return","Vol","Sharp_Ratio"]
    
    for k in range(len(Weight_matrice)):
        Weight_matrice[k] = sort_row(Weight_matrice[k])
    
    return ret_vol_sr_matrice,Weight_matrice

# Fonction de test de la matrice d'allocation : 
# Input_Type : DataFrame
# Output_Type : String 
def test_weight_allocated(Dataframe):
    
    sum_list = []
    
    for k in range (len(Dataframe)):
        sum_list.append(np.sum(np.abs(Dataframe[k])))
        
    i = 0
    for data in sum_list:
        if data >1 : 
            i = i + 1
    if i > 0:
        print("Allocation valide")
    else : 
        print("allocation non valide")
        
# Construcion de la fonction MAR : 
# Input_Type : Matrice, Float, DataFrame
# Output_Type : Matrice
def get_MAR(weight_matrice,mar_value,portefeuille):
    
    matrice_weight = weight_matrice.copy()
    
    super_liste = []
    
    for value1,value2 in zip(matrice_weight,portefeuille):
        
        liste = []
        
        for k,ticker in zip(range(len(value1)),value2.columns):
            liste.append((value2[ticker].mean() * value1[k]) * 252)
            
        super_liste.append(liste)
        
    super_liste = pd.DataFrame(super_liste)
    
    for k in range(len(super_liste)):
        j = 0
        
        for value1,value2 in zip(super_liste.loc[k][1:],matrice_weight[k][1:]):
            if value1 <= mar_value:
                matrice_weight[k][0] = matrice_weight[k][0] + value2
                matrice_weight[k][1:][j] = 0 
                
            j = j + 1
            
    return matrice_weight

# Préparation des portefeuilles, les rends compatibles avec l'optmimiseur : 
# Input_Type : DataFrame
# Output_Type : DataFrame
def prepare_portefeuille_2(Portefeuille,backtest_starting_date,backtest_ending_date):
    portefeuille = Portefeuille.copy()
    portefeuille = portefeuille[2:]
    portefeuille.rename(columns = {"Unnamed: 0" : 'Date'},inplace = True)

    if "Date" in portefeuille.columns:
        portefeuille.set_index("Date",inplace = True)
        
    portefeuille = portefeuille[backtest_starting_date:backtest_ending_date]
    return portefeuille

# Recherche des dates de rebalancement, localisation des dates de fin de calcul des nouveaux poids pour chaques périodes : 
# Input_Type : DataFrame 
# Output_Type : Date_List
def get_allocations_dates(DataFrame) : 
    
    liste_ = []
    
    for value in DataFrame:
        liste_.append(value.index[-1])
        
    return liste_

# Fonction d'écriture du noms de la matrice d'allocation :
# Input_Type : Matrice,Int,Float
# Output_Type : String
def get_weight_name(matrice,lenght,mar_value):
    return  TODAY + "_" + matrice + " {}".format(lenght) + ' MAR ' + " {}".format(mar_value) + ".xlsx"

# Fontion de produit cumulatif : 
# Input_Type : Int_List
# Output_Type : Float
def produit(liste):
    _produit = 1
    for i in liste:
        _produit = _produit * (1 + i)
    return _produit

# Fonction pour récupérer le Ratio de Sharp annuel moyen :
# Input_Type : Matrice 
# Outpu_Type : Float
def get_stats(matrice): 
    return round(matrice["Sharp_Ratio"].mean(),2)

# Cette fonction prends en input un portefeuille et renvoie le taux Eonia capitalisé en daily :
# Input_Type : DataFrame
# Output_Type : Float_List
def Get_Eonia_Rate(portefeuille):
    
    portefeuille_for_Eonia = portefeuille.copy()
    portefeuille_for_Eonia.reset_index(inplace = True)
    portefeuille_for_Eonia = portefeuille_for_Eonia[['Date','EONIA Index']]
    portefeuille_for_Eonia.loc[:,('Days_count')] = portefeuille_for_Eonia.loc[:,('Date')].diff().dt.days
    portefeuille_for_Eonia.loc[0,('Days_count')] = 0  
    
    return_list = [] 
    Eonia_capitalisation = [] 
    return_list.append(portefeuille_for_Eonia["EONIA Index"][0])
    
    for j in range (1,len(portefeuille_for_Eonia)):
        return_list.append(((portefeuille_for_Eonia.loc[j,('EONIA Index')]/100) * portefeuille_for_Eonia.loc[j,('Days_count')]) /365)
        Eonia_capitalisation.append(produit(return_list))
        
    return (Eonia_capitalisation)

# Cette fonction prends une liste de dataFrame et renvoie une liste de taux correspondants à chacuns d'entre eux :
# Input_Type : DataFrame_List 
# Output_Type : Float_List_List
def Get_Eonia_Rate_for_Sharp(portefeuille_list):
    Eonia_Rate = []

    for portefeuille in portefeuille_list:
    
        portefeuille_for_Eonia = portefeuille.copy()
        portefeuille_for_Eonia.reset_index(inplace = True)
        portefeuille_for_Eonia = portefeuille_for_Eonia[['Date','EONIA Index']]
        portefeuille_for_Eonia.loc[:,('Days_count')] = portefeuille_for_Eonia.loc[:,('Date')].diff().dt.days
        portefeuille_for_Eonia.loc[0,('Days_count')] = 0  
        
        Total_capitalisation = []
        return_list = [] 
        Eonia_capitalisation = [] 
    
        for j in range (len(portefeuille_for_Eonia)):
            return_list.append(((portefeuille_for_Eonia.loc[j,('EONIA Index')]/100) * portefeuille_for_Eonia.loc[j,('Days_count')]) /365)
            Eonia_capitalisation.append(produit(return_list))
       
        Total_capitalisation.append(Eonia_capitalisation)
        Eonia_Rate.append((Total_capitalisation[0][-1]/Total_capitalisation[0][0])**((365/len(portefeuille_for_Eonia))-1) - 1)
        
    return (Eonia_Rate)

# Obtention des equity_curve de chacuns des portefeuilles :
# Input_Type : DataFrame_List,DataFrame,Matrice,String_List,int
# Output_Type : Float_List 
def get_valorisation(Portefeuille_list,Portefeuille_before_filled,weight_matrice,Ticker_list,window_lenght):
    
    l = 0 
    portefeuille_list = []
    Index_List_Filtered_flat = Ticker_list[1:]
    # matrice des parts Index : 
    matrice_list_parts = np.zeros((len(weight_matrice),len(Index_List_Filtered_flat)))
    # initialisation de la liste de parts EONIA : 
    matrice_list_parts_Eonia = np.zeros((len(weight_matrice),1))
    # Valorisation Eonia : 
    Amount = []
    Amount.append(10000)

    # utilisation de cette liste pour valorisation Eonia sur portefeuille  : 
    # préparation du portefeuille : 
    # Utiliser la capitalisation Eonia calculée plus haut : 

    for (portefeuille,k) in zip (Portefeuille_list, range(len(weight_matrice))):
        matrice_list_parts_Eonia[k] = Amount[k] * np.abs(weight_matrice.iloc[k][0])
        portefeuille_for_Eonia = portefeuille.copy()
        portefeuille_for_Eonia.reset_index(inplace = True)
        portefeuille_for_Eonia = portefeuille_for_Eonia[['Date','EONIA Index']]
        portefeuille_for_Eonia.loc[:,('Days_count')] = portefeuille_for_Eonia.loc[:,('Date')].diff().dt.days
        portefeuille_for_Eonia.loc[0,('Days_count')] = 0  
        
        # maintenant que le portefeuille est prèt : calcul de la valorisation : 

        Total_capitalisation = [] 
        return_list = [] 
        Eonia_capitalisation = [] 
        Eonia_capitalisation.append(matrice_list_parts_Eonia[k])
    
        for j in range (len(portefeuille_for_Eonia) - 1):
            return_list.append(((portefeuille_for_Eonia.loc[j,('EONIA Index')]/100) * portefeuille_for_Eonia.loc[j,('Days_count')]) /360)
            Eonia_capitalisation.append(produit(return_list) * matrice_list_parts_Eonia[k])
    
        Total_capitalisation.append(Eonia_capitalisation)
    
    
        # Création du portefeuille de valorisation : 
    
        portefeuille_final = pd.DataFrame(Total_capitalisation)
        portefeuille_final = portefeuille_final.T
        portefeuille_final.columns = ["Eonia Capitalisé"]
        portefeuille_final.set_index(portefeuille_for_Eonia.Date,inplace = True)
    
    
        # On passe à la valorisation du reste du portefeuille et création de la liste de portefeuille valorisés : 

        Nombre_de_parts = []

        for weight,ticker in zip(weight_matrice.iloc[k][1:],Index_List_Filtered_flat):
                Nombre_de_parts.append((Amount[k] * weight) / Portefeuille_before_filled[ticker][l]) # Gérer deux listes: 
                
        matrice_list_parts[k,:] = Nombre_de_parts
    
        l = l + window_lenght
    
        portefeuille_for_valorisation = portefeuille.copy()
        portefeuille_for_valorisation.drop(columns = ["EONIA Index"], inplace = True)
        portefeuille_for_valorisation = portefeuille_for_valorisation * matrice_list_parts[k,:]
        portefeuille_final = pd.concat([portefeuille_final,portefeuille_for_valorisation],axis = 1)
        
        portefeuille_final["Capitalisation"] = portefeuille_final.sum(axis = 1)
        Amount.append(portefeuille_final["Capitalisation"][-1])
        portefeuille_list.append(portefeuille_final)
        matrice_list_parts_Eonia[k] = Amount[k] * np.abs(weight_matrice.iloc[k][0])
    
    # On cherche maintenant l'equity curve du portefeuille G : 
    equity_list = []
    for portefeuille in portefeuille_list:
        for i in portefeuille["Capitalisation"]:
            equity_list.append(i)
        
    return equity_list

# Fonction de calcul de return : 
# Input_Type : Float_List
# Outut_Type : Float
def get_return(equity_curve):
    return (equity_curve.iloc[-1]/equity_curve.iloc[0] - 1) * 100

# Fonction qui permet de créer un differentiel sous_jacents/allocation : 
# Input_type : DataFrame_List,Float_List,Float_List (Double)
# Output_Type : Float_List 
# Fonction a foutre en base 100 pour rendement relatif : 
def get_data_for_differentiel(ouput,equity_curve,Bound_list):
    
    scaler = MinMaxScaler()
    Output = pd.concat(ouput[3]) # Output[3] == Portefeuille_List 
    liste = []
    for value in Bound_list:
        liste.append(value[1])
        
    Output = Output * liste
    Output = scaler.fit_transform(Output)
    Output = pd.DataFrame(Output).sum(axis=1)
    Output.index = equity_curve.index
    
    return Output

# Fonction qui permet de caller en annuel une durée : 
# Input_Type : Date,Date
# Output_Type : Float
def get_yearly_duration(Starting_Date,Ending_Date,window_backtest):
    
    start = Starting_Date.date()
    end = Ending_Date.date()
    output = np.busday_count(start,end) - window_backtest
    
    return round(output/261,3) 

# Fonction de calcul du time_to_recovery : 
# Input_Type : Float_Liste 
# Output_Type : Float
# Idée : Faire un scan sur le drawdown et compter à partir d'une perte supérieure à 1.5% 
def get_time_to_recovery(drawdown_curve):
    
    liste = [] 
    nombre = 0 
    
    drawdown_curve["compteur"] = drawdown_curve[drawdown_curve.columns[0]]
    
    for k in range (len(drawdown_curve)):
        if drawdown_curve[drawdown_curve.columns[0]][k]*100<=-1.5:
            drawdown_curve["compteur"][k] = 1
            
        else : 
             drawdown_curve["compteur"][k] = 0
            
    for k in range (len(drawdown_curve["compteur"])-1):
        if drawdown_curve["compteur"][k] == 1 and drawdown_curve["compteur"][k+1] == 1:
            nombre+=1
            
        if drawdown_curve["compteur"][k] == 1 and drawdown_curve["compteur"][k+1] == 0:
            nombre = 0
            
        liste.append(nombre)
        
    liste = pd.DataFrame(liste)
    drawdown_curve.drop(columns = "compteur",inplace = True)
    
    return round(liste.max()/252,2)

# Fonction de calcul des statistiques réalisées: 
# Input_Type : Matrice, DataFrame
# Ouput_Type : DataFrame
def get_realized_ret_vol_sr(weight_matrice,portefeuilles):
    weight_matrice = weight_matrice.iloc[:-1]
    ret_vol_sr_matrice = np.zeros((len(portefeuilles),3))
    i = 0
    for k in range (len(weight_matrice)):
        
        ret = np.sum(pd.DataFrame(portefeuilles[k]).diff().mean() * weight_matrice.iloc[k]) * 252
        vol = np.sqrt(np.dot(weight_matrice.iloc[k].T, np.dot(pd.DataFrame(portefeuilles[k]).diff().cov() * 252, weight_matrice.iloc[k])))
        sr = ret/vol
        ret_vol_sr_matrice[i:] = [ret,vol,sr]
        i = i+1
        
    return pd.DataFrame(ret_vol_sr_matrice)

# Calcul du DD : 
# Input_Type : Float_List
# Output_Type : Float_List
def DD(Equity):
    previous_peaks = Equity.cummax()
    drawdown = (Equity-previous_peaks) / previous_peaks
    return drawdown 

# Calcul du Max_DD : 
# Input_Type : Float_List
# Output_Type : Float
def Max_DD(equity):
    return round(DD(equity).min()*100)

# Calcul du ratio de Sortino : 
# Définition de la fonction de calcul du ratio de sortino : 
# Input_Type : Float_List,Float
# Output_Type : Float
# Ré écrire la fonction pour fitter avec window_lenght : 
def Sortino_Ratio(real_return,Expected_Return):
    
    start_index = 0
    end_index = 12
    annual_duration = 12
    real_return_monthly_annually = []
    mean_return = np.mean(real_return)
    
    for k in range (int(len(real_return)/12)):
        real_return_monthly_annually.append(real_return[start_index:end_index])
        start_index = start_index + annual_duration
        end_index = end_index + annual_duration 
    
    return_annual = []
    for année in real_return_monthly_annually:
        return_annual.append(année.mean())
    
    real_return_monthly_annually_mean = []
    for année in real_return_monthly_annually:
        real_return_monthly_annually_mean.append(année.mean())   
    real_return_monthly_annually_mean = pd.DataFrame(real_return_monthly_annually_mean)
    
    list_for_Sortino = []
    for mean in real_return_monthly_annually_mean[0]:
        if mean <= 0.05:
            list_for_Sortino.append(mean)   
        
    list_Sortino = []
    for element in list_for_Sortino:
        element_ = (Expected_Return - element)**2
        list_Sortino.append(element_)
        
    sigma_sortino = np.sqrt(np.sum(list_Sortino)/len(real_return_monthly_annually_mean))
    Ratio_Sortino = (mean_return - Expected_Return)/sigma_sortino
    
    return (round(Ratio_Sortino,2))

# Calcul du ratio de Calmar : 
# Input_Type : Float,Float
# Output_Type : Float
def Calmar_ratio(real_return,max_DD):
    return round(real_return/max_DD,2)

# Calcul du return de la stratégie : 
# Input_Type : Float_List
# Output_Type : Float
def get_strategy_return(equity_curve):
        return round(equity_curve.iloc[-1]/equity_curve.iloc[0],1)
    
# Calcul du return annualisé de la stratégie : 
# Input_Type : Float,Float
# Output_Type : Float    
def Get_Annualized_Return(Return,duration):
    return round((round(Return,2)**(1/duration)-1)*100,2)   

# Calcul du return de la stratégie : 
# Input_Type : DataFrame_List, Matrice_List
# Output_Type : Date_Indexed_DataFrame
def get_realized_returns(output,weight_matrice):
    liste = [] 
    weight_matrice.columns = output[3][0].columns

    for value,k in zip (output[3],range(len(output[3]))):
        liste.append(value*weight_matrice[:-1].iloc[k])
        
    super_return_list = []
    for value,k in zip(liste,range(len(output[3]))):
        return_liste = []
        
        for column in value.columns:
            return_liste.append(round(((value[column][-1]/value[column][0])-1)*100,3))
            
        return_liste = return_liste[1:] 
        return_liste = pd.DataFrame(return_liste).replace(np.nan, 0)
        return_liste = round(return_liste * pd.DataFrame(weight_matrice[:-1].iloc[k][1:].values),3)
        super_return_list.append(return_liste)
        
    super_return_list = pd.concat(super_return_list,axis = 1).T 
    super_return_list = pd.DataFrame(super_return_list)
    super_return_list.index = weight_matrice.index[:-1]
    super_return_list.columns = weight_matrice.columns[1:]
    
    return super_return_list

# Calcul du return des sous jacents de la stratégie : 
# Input_Type : Matrice_Row
# Output_Type : Int_Array
def set_to_zero(liste_):
    liste = []
    for value in liste_:
        if value !=0:
            liste.append(1)
        else:
            liste.append(0)
    return liste 

# Calcul du return des sous jacents de la stratégie : Poids fixés à un si non nuls
# Input_Type : DataFrame_List, Matrice_List
# Output_Type : Date_Indexed_DataFrame
def get_returns(ouput,weight_matrice):
    
    liste = [] 
    for value in (ouput[3]):
        liste.append(value)
        
    super_return_list = []
    for value,k in zip(liste,range(len(ouput[3]))):
        return_liste = []
        
        for column in value.columns:
            return_liste.append(round(((value[column][-1]/value[column][0])-1)*100,3))
            
        for data in weight_matrice[:-1].iloc[k][1:]:
            if data !=0:
                data=1
                
        return_liste = return_liste[1:]
        return_liste = pd.DataFrame(return_liste).replace(np.nan, 0)
        return_liste = round(return_liste * pd.DataFrame(set_to_zero(weight_matrice[:-1].iloc[k][1:])),3)
        super_return_list.append(return_liste)
        
    super_return_list = pd.concat(super_return_list,axis = 1).T 
    super_return_list = pd.DataFrame(super_return_list)
    super_return_list.index = weight_matrice.index[:-1]
    super_return_list.columns = weight_matrice.columns[1:]
    
    return super_return_list

def progress_bar(progress,total):
    
    percent = 100 * (progress/float(total))
    bar = '█' * int(percent) + '-' * (100 - int(percent))
    print(f"\r|{bar}| {percent:.2f}%", end = "\r")


# Fonction d'optimisation des portefeuilles, regroupe l'ensemble des fonctions définies jusqu'ici : 
# Output : Ensemble des graphes regroupants les résultats.
def optimize(window_for_backtest,MAR_value):
    import datetime
    
    if MAR_value == 0 and window_for_backtest != 0:
        print("Set de paramêtres incompatibles, veuillez recommencer")
        
        
    Noms_Statistiques = []
    Noms_Drawdowns = []
    Noms_Equity_Curves = []
    Table_de_correspondance = []
    
    k = 0 
    
    for (portefeuille,portefeuille_2) in zip (directory_list,portefeuille_name_list):
        
        wb = load_workbook(directory_path + portefeuille_2)
        Portefeuille = pd.read_excel(portefeuille)
        # Lecture des fichiers Excel : 
        
        # Get Basis Curency :
        coord = next(wb.defined_names["_BasisCcy"].destinations)
        basis = wb[coord[0]][coord[1]].value

        # Listes des Tickers :
        ws, range_= next(wb.defined_names["Ticker_list"].destinations)
        Ticker_List = [[cell.value for cell in row] for row in wb[ws][range_]]

        Ticker_List_flat = []
        for i in Ticker_List:
            for j in i:
                if j != None:
                    Ticker_List_flat.append(j)

        # Récupération du format pour les Back-tests :
        coord = next(wb.defined_names["window_lenght"].destinations)
        window_lenght = wb[coord[0]][coord[1]].value
            
        # Listes des Asset_Bounds :
        ws_bound, range_bound= next(wb.defined_names["Min_Max_list"].destinations)
        Bound_List = [[cell.value for cell in row] for row in wb[ws_bound][range_bound]]
        
        Bound_List_filtered = []
        
        for value in Bound_List:
            if value != [None,None]:
                Bound_List_filtered.append(value)

        """# Starting Date : 
        coord = next(wb.defined_names["Ending_Date"].destinations)
        Ending_Date = wb[coord[0]][coord[1]].value
        
        # Ending Date : 
        coord = next(wb.defined_names["Starting_Date"].destinations)
        Starting_Date = wb[coord[0]][coord[1]].value"""
        
        # Vol Max : 
        coord = next(wb.defined_names["Vol_Max"].destinations)
        Vol_MAX = wb[coord[0]][coord[1]].value

        # Backtest_starting_date : 
        backtest_starting_date = datetime.datetime(année_début, mois_début, jour_début)
        backtest_starting_date = backtest_starting_date + BDay(1)
        backtest_starting_date = datetime.datetime(int(str(backtest_starting_date)[:4]), int(str(backtest_starting_date)[5:7]), int(str(backtest_starting_date)[8:10]))

        # Backtest_ending_date : 
        backtest_ending_date = datetime.datetime(année_fin,mois_fin,jour_fin)
        backtest_ending_date = backtest_ending_date + BDay(1)
        backtest_ending_date = datetime.datetime(int(str(backtest_ending_date)[:4]), int(str(backtest_ending_date)[5:7]), int(str(backtest_ending_date)[8:10]))
        
        Portefeuille = prepare_portefeuille_2(Portefeuille,backtest_starting_date,backtest_ending_date)
        Portefeuille_lenght = len(Portefeuille.columns)

        # MAR : Si le paramêtre MAR est fixé à 0 : Alors optimisation selons les deux MAR spécifiés sur les fichiers Excel.
        if MAR_value == 0 and window_for_backtest == 0:
                        
            coord = next(wb.defined_names["window_backtest_93"].destinations)
            window_backtest_93 = wb[coord[0]][coord[1]].value
            
            coord = next(wb.defined_names["window_backtest_63"].destinations)
            window_backtest_63 = wb[coord[0]][coord[1]].value
            
            window_backtest_list = [window_backtest_63,window_backtest_93]
            
            coord = next(wb.defined_names["MAR_63"].destinations)
            MAR_63 = wb[coord[0]][coord[1]].value       
            
            coord = next(wb.defined_names["MAR_93"].destinations)
            MAR_93 = wb[coord[0]][coord[1]].value 
                    
            Liste_MAR = [MAR_63,MAR_93]
            
            for (MAR,j) in zip(Liste_MAR,range(len(Liste_MAR))):
                
                window_backtest = window_backtest_list[j]

                Output =  prepare_portefeuille(Portefeuille,window_backtest,window_lenght,backtest_starting_date,backtest_ending_date)         
                Log_return_portefeuille_liste = Output[-1]
            

                ret_vol_sr_matrice,Weight_matrice = Make_Markowitz_Optimisation(Log_return_portefeuille_liste,Vol_MAX,Bound_List_filtered,Portefeuille_lenght,max_short_trehsold,window_lenght)

                Weight_matrice = get_MAR(Weight_matrice,MAR,Log_return_portefeuille_liste)
                Weight_matrice = pd.DataFrame(Weight_matrice)
                Weight_matrice.columns = Ticker_List_flat
                Weight_matrice.index = get_allocations_dates(Log_return_portefeuille_liste)

                realized_returns = get_realized_returns(Output,Weight_matrice)

                realized_returns.to_excel(Liste_for_database_mngt[k] + "/Realized returns" + '/' + '{}'.format(TODAY)  + "{}".format(portefeuille[-16:-5]) +  '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' +'{}'.format(MAR) + '.xlsx')
                plt.figure(figsize = (25,13))
                plt.plot(realized_returns,label = realized_returns.columns)
                plt.legend(loc = 0)
                plt.title('{}'.format(TODAY) + "{}".format(portefeuille) + ' Realized returns' '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' +'{}'.format(MAR))
                plt.savefig(Liste_for_database_mngt[k] + "/Realized returns" + '/'  + '{}'.format(TODAY) + "{}".format(portefeuille[-16:-5]) + ' Realized returns_' + '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' + '{}'.format(MAR) + '.png')

                returns = get_returns(Output,Weight_matrice)

                plt.figure(figsize = (25,13))
                plt.plot(returns,label = returns.columns)
                plt.legend(loc = 0)
                plt.title('{}'.format(TODAY) + "{}".format(portefeuille) + ' Absolute returns' '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' +'{}'.format(MAR))
                plt.savefig(Liste_for_database_mngt[k] + "/Absolute returns" + '/'  + '{}'.format(TODAY) + "{}".format(portefeuille[-16:-5]) + ' Absolute returns' + ' {}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' + '{}'.format(MAR) + '.png')

                plt.figure(figsize = (25,13))
                plt.stackplot(Weight_matrice.index,[Weight_matrice[Weight_matrice.columns[i]] for i in range(1, len(Weight_matrice.columns))],Weight_matrice[Weight_matrice.columns[0]],labels = Weight_matrice.columns[1:],colors = color_liste[-len(Weight_matrice.columns):])
                plt.legend(loc=2)
                plt.title("{}".format(TODAY) + "{}".format(portefeuille[-16:-5]) + "_" +   "Allocation" + "_" + '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' +'{}'.format(MAR))
                plt.savefig(Liste_for_database_mngt[k] + "/Profils allocation" + '/'  +"{}".format(TODAY) + "{}".format(portefeuille[-16:-5]) + " Profil Allocation" + "_" + '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght) + '_' + 'MAR' +'{}'.format(MAR) + '.png')

                Weight_matrice.to_excel(Liste_for_database_mngt[k] + "/Weight matrices" + '/'  + "{} ".format(TODAY) + "{} ".format(portefeuille[-16:-5]) + get_weight_name("Weight_matrice",window_backtest,MAR))

                # Construction des Equity_curve : 
                equity_list = get_valorisation(Output[3],Output[2],Weight_matrice,Ticker_List_flat,window_lenght)
                equity_list = pd.DataFrame(equity_list)
                equity_list.set_index(Output[2].index, inplace=True)

                Real_Return = get_return(equity_list)[0]

                # Plotting des Equity_curve :
                plt.figure(figsize = (25,13))
                plt.plot(equity_list,label = "Equity_Curve" + '{}'.format(portefeuille[-16:-5]))
                plt.legend()
                plt.title("Equity_curve du Portefeuille")
                equity_list.to_csv(Liste_for_database_mngt[k] + "/Equity Curves" + '/'  +'{}'.format(TODAY) + " {}".format(portefeuille[-16:-5]) +  " equity Curve"+ ' {}'.format(window_backtest) +' MAR' + ' {}'.format(MAR) +".csv")

                # Calcul de la performance absolu des lignes du portefeuilles pour pouvoir comparer avec l'equity curve obtenue : 
                perf = get_data_for_differentiel(Output,equity_list,Bound_List_filtered)
                fig, ax1 = plt.subplots(figsize = (25,13))
                ax1.plot(perf,color = 'red')
                ax2 = ax1.twinx()
                ax2.plot(equity_list,label = "Equity Curve")
                plt.title("{}".format(TODAY) + "Performance absolue des lignes VS Equity Curve" + " Markowitz classique" + "{}".format(MAR))
                plt.legend()
                plt.savefig(Liste_for_database_mngt[k] + "/Performances VS Equity Curves" + '/'  +"{}".format(TODAY) + "{}".format(portefeuille[-16:-5]) + '{}'.format(window_backtest) + " Performance absolue des lignes VS Equity Curve" + "{}".format(MAR) + ".png")

                 # Calcul et affichage du Max Drawdown : 
                previous_peaks = equity_list.cummax()
                drawdown =  (equity_list-previous_peaks) / previous_peaks

                plt.figure(figsize = (25,13))
                plt.plot(drawdown,label = "Drawdown")
                plt.legend()
                plt.title('Drawdown du portefeuille')
                plt.savefig(Liste_for_database_mngt[k] + "/Drawdowns" + '/' +'{}'.format(TODAY) + " Drawdown du Portefeuille" + "{}".format(portefeuille[-16:-5]) + '{}'.format(window_backtest) +'MAR' +'{}'.format(MAR) +'.png')
                drawdown.to_csv(Liste_for_database_mngt[k] + "/Drawdowns csv" + '/' +'{}'.format(TODAY) + " Drawdown du Portefeuille" + " {}".format(portefeuille[-16:-5]) + ' {}'.format(window_backtest) +' MAR' +' {}'.format(MAR) +'.csv')

                Real_Return = get_return(equity_list)[0]
                total_return = round(get_strategy_return(equity_list)[0],3)
                duration  = get_yearly_duration(backtest_starting_date,backtest_ending_date,window_backtest)

                 # Création du Tableaux regroupant les statistiques : 
                Annualized_return = Get_Annualized_Return(total_return,duration)
                MaX_DD = Max_DD(equity_list)[0]
                returns = (equity_list.iloc[-1]/equity_list.iloc[0]) -1 
                Calmar_Ratio = Annualized_return/MaX_DD
                Mean_Annualized_vol = get_realized_ret_vol_sr(Weight_matrice,Output[3])[1].mean()
                Mean_Annualized_return = get_realized_ret_vol_sr(Weight_matrice,Output[3])[0].mean()
                Mean_Sharp_ratio = Mean_Annualized_return/Mean_Annualized_vol
                Annualized_time_to_recovery = get_time_to_recovery(drawdown)[0]
                stats_liste = [(total_return-1) * 100 ,Mean_Annualized_return,Mean_Annualized_vol,MaX_DD,Calmar_Ratio,Mean_Sharp_ratio,Annualized_time_to_recovery]

                liste = []
                for value in stats_liste:
                    liste.append(round(value,3))

                liste = pd.DataFrame(stats_liste).T
                liste.columns = ["total_return : Start to Date %","Mean Annualizedreturn","Mean Annualized Vol","MaX Drawdown","Calmar Ratio","Sharp Ratio","Annualized time to recovery"]
                infos = ["Statistiques de la stratégie"]
                liste.index = infos
                liste.to_excel(Liste_for_database_mngt[k] + "/Statistiques" + '/'  + '{}'.format(TODAY) + "{}".format(portefeuille[-16:-5]) + ' Statistiques_Portefeuilles' +' {}'.format(window_backtest) + ' {}'.format(window_lenght)  +'{}'.format(MAR) + '.xlsx')
                
                progress_bar(k,len(Liste_for_database_mngt))

                Noms_Statistiques.append(Liste_for_database_mngt[k] + "/Statistiques" + '/'  + '{}'.format(TODAY) + "{}".format(portefeuille[-16:-5]) + ' Statistiques_Portefeuilles' +' {}'.format(window_backtest) + ' {}'.format(window_lenght)  +'{}'.format(MAR) + '.xlsx')
                Noms_Drawdowns.append(Liste_for_database_mngt[k] + "/Drawdowns csv" + '/' +'{}'.format(TODAY) + " Drawdown du Portefeuille" + "{}".format(portefeuille[-16:-5]) + '{}'.format(window_backtest) +'MAR' +'{}'.format(MAR) +'.csv')
                Noms_Equity_Curves.append(Liste_for_database_mngt[k] + "/Equity Curves" + '/'  +'{}'.format(TODAY) + "{}".format(portefeuille[-16:-5]) +  " equity Curve"+ '{}'.format(window_backtest) +'MAR' + '{}'.format(MAR) + ".csv")
                k = k + 1                       
            
            Table_de_correspondance = Noms_Statistiques
            Table_de_correspondance = pd.DataFrame(Table_de_correspondance)
            Table_de_correspondance["Drawdowns"] = Noms_Drawdowns
            Table_de_correspondance["Equity Curves"] = Noms_Equity_Curves

            return Table_de_correspondance

        else: 
            
            window_backtest = int(window_for_backtest)   
            MAR = MAR_value
        
                
            Output =  prepare_portefeuille(Portefeuille,window_backtest,window_lenght,backtest_starting_date,backtest_ending_date)         
            Log_return_portefeuille_liste = Output[-1]
            Portefeuille = prepare_portefeuille_2(Portefeuille,backtest_starting_date,backtest_ending_date)
            Portefeuille_lenght = len(Portefeuille.columns)

            ret_vol_sr_matrice,Weight_matrice = Make_Markowitz_Optimisation(Log_return_portefeuille_liste,Vol_MAX,Bound_List_filtered,Portefeuille_lenght,max_short_trehsold,window_lenght)

            Weight_matrice = get_MAR(Weight_matrice,MAR,Log_return_portefeuille_liste)
            Weight_matrice = pd.DataFrame(Weight_matrice)
            Weight_matrice.columns = Ticker_List_flat
            Weight_matrice.index = get_allocations_dates(Log_return_portefeuille_liste)

            realized_returns = get_realized_returns(Output,Weight_matrice)

            realized_returns.to_excel(Liste_for_database_mngt[k] + "/Realized returns" + '/' + '{}'.format(TODAY)  + "{}".format(portefeuille[-16:-5]) +  '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' +'{}'.format(MAR) + '.xlsx')
            plt.figure(figsize = (25,13))
            plt.plot(realized_returns,label = realized_returns.columns)
            plt.legend(loc = 0)
            plt.title('{}'.format(TODAY) + "{}".format(portefeuille) + ' Realized returns' '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' +'{}'.format(MAR))
            plt.savefig(Liste_for_database_mngt[k] + "/Realized returns" + '/'  + '{}'.format(TODAY) + "{}".format(portefeuille[-16:-5]) + ' Realized returns_' + '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' + '{}'.format(MAR) + '.png')

            returns = get_returns(Output,Weight_matrice)

            plt.figure(figsize = (25,13))
            plt.plot(returns,label = returns.columns)
            plt.legend(loc = 0)
            plt.title('{}'.format(TODAY) + "{}".format(portefeuille) + ' Absolute returns' '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' +'{}'.format(MAR))
            plt.savefig(Liste_for_database_mngt[k] + "/Absolute returns" + '/'  + '{}'.format(TODAY) + "{}".format(portefeuille[-16:-5]) + ' Absolute returns' + ' {}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' + '{}'.format(MAR) + '.png')

            plt.figure(figsize = (25,13))
            plt.stackplot(Weight_matrice.index,[Weight_matrice[Weight_matrice.columns[i]] for i in range(1, len(Weight_matrice.columns))],Weight_matrice[Weight_matrice.columns[0]],labels = Weight_matrice.columns[1:],colors = color_liste[-len(Weight_matrice.columns):])
            plt.legend(loc=2)
            plt.title("{}".format(TODAY) + "{}".format(portefeuille[-16:-5]) + "_" +   "Allocation" + "_" + '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght)+ '_' + 'MAR' +'{}'.format(MAR))
            plt.savefig(Liste_for_database_mngt[k] + "/Profils allocation" + '/'  +"{}".format(TODAY) + "{}".format(portefeuille[-16:-5]) + " Profil Allocation" + "_" + '{}'.format(window_backtest) + "_"  + '{}'.format(window_lenght) + '_' + 'MAR' +'{}'.format(MAR) + '.png')

            Weight_matrice.to_excel(Liste_for_database_mngt[k] + "/Weight matrices" + '/'  + "{} ".format(TODAY) + "{} ".format(portefeuille[-16:-5]) + get_weight_name("Weight_matrice",window_backtest,MAR))

            # Construction des Equity_curve : 
            equity_list = get_valorisation(Output[3],Output[2],Weight_matrice,Ticker_List_flat,window_lenght)
            equity_list = pd.DataFrame(equity_list)
            equity_list.set_index(Output[2].index, inplace=True)

            Real_Return = get_return(equity_list)[0]

            # Plotting des Equity_curve :
            plt.figure(figsize = (25,13))
            plt.plot(equity_list,label = "Equity_Curve" + '{}'.format(portefeuille[-16:-5]))
            plt.legend()
            plt.title("Equity_curve du Portefeuille")
            equity_list.to_csv(Liste_for_database_mngt[k] + "/Equity Curves" + '/'  +' {}'.format(TODAY) + " {}".format(portefeuille[-16:-5]) +  " equity Curve"+ ' {}'.format(window_backtest) +' MAR' + ' {}'.format(MAR) +".csv")

            # Calcul de la performance absolu des lignes du portefeuilles pour pouvoir comparer avec l'equity curve obtenue : 
            perf = get_data_for_differentiel(Output,equity_list,Bound_List_filtered)
            fig, ax1 = plt.subplots(figsize = (25,13))
            ax1.plot(perf,color = 'red')
            ax2 = ax1.twinx()
            ax2.plot(equity_list,label = "Equity Curve")
            plt.title("{}".format(TODAY) + "Performance absolue des lignes VS Equity Curve" + " Markowitz classique" + "{}".format(MAR))
            plt.legend()
            plt.savefig(Liste_for_database_mngt[k] + "/Performances VS Equity Curves" + '/'  +"{}".format(TODAY) + "{}".format(portefeuille[-16:-5]) + '{}'.format(window_backtest) + " Performance absolue des lignes VS Equity Curve" + "{}".format(MAR) + ".png")

             # Calcul et affichage du Max Drawdown : 
            previous_peaks = equity_list.cummax()
            drawdown =  (equity_list-previous_peaks) / previous_peaks

            plt.figure(figsize = (25,13))
            plt.plot(drawdown,label = "Drawdown")
            plt.legend()
            plt.title('Drawdown du portefeuille')
            plt.savefig(Liste_for_database_mngt[k] + "/Drawdowns" + '/' +'{}'.format(TODAY) + " Drawdown du Portefeuille" + "{}".format(portefeuille[-16:-5]) + '{}'.format(window_backtest) +'MAR' +'{}'.format(MAR) +'.png')
            drawdown.to_csv(Liste_for_database_mngt[k] + "/Drawdowns csv" + '/' +' {}'.format(TODAY) + " Drawdown du Portefeuille" + " {}".format(portefeuille[-16:-5]) + ' {}'.format(window_backtest) +' MAR' +' {}'.format(MAR) +'.csv')

            Real_Return = get_return(equity_list)[0]
            total_return = round(get_strategy_return(equity_list)[0],3)
            duration  = get_yearly_duration(backtest_starting_date,backtest_ending_date,window_backtest)

            # Création du Tableaux regroupant les statistiques : 
            Annualized_return = Get_Annualized_Return(total_return,duration)
            MaX_DD = Max_DD(equity_list)[0]
            returns = (equity_list.iloc[-1]/equity_list.iloc[0]) -1 
            Calmar_Ratio = Annualized_return/MaX_DD
            Mean_Annualized_vol = get_realized_ret_vol_sr(Weight_matrice,Output[3])[1].mean()
            Mean_Annualized_return = get_realized_ret_vol_sr(Weight_matrice,Output[3])[0].mean()
            Mean_Sharp_ratio = Mean_Annualized_return/Mean_Annualized_vol
            Annualized_time_to_recovery = get_time_to_recovery(drawdown)[0]
            stats_liste = [(total_return-1) * 100 ,Mean_Annualized_return,Mean_Annualized_vol,MaX_DD,Calmar_Ratio,Mean_Sharp_ratio,Annualized_time_to_recovery]

            liste = []
            for value in stats_liste:
                liste.append(round(value,3))

            liste = pd.DataFrame(stats_liste).T
            liste.columns = ["total_return : Start to Date %","Mean Annualizedreturn","Mean Annualized Vol","MaX Drawdown","Calmar Ratio","Sharp Ratio","Annualized time to recovery"]
            infos = ["Statistiques de la stratégie"]
            liste.index = infos
            
            liste.to_excel(Liste_for_database_mngt[k] + "/Statistiques" + '/'  + ' {}'.format(TODAY) + " {}".format(portefeuille[-16:-5]) + ' Statistiques_Portefeuilles' +' {}'.format(window_backtest) + ' {}'.format(window_lenght)  +' {}'.format(MAR) + '.xlsx')
            progress_bar(k,len(Liste_for_database_mngt))

            
            Noms_Statistiques.append(Liste_for_database_mngt[k] + "/Statistiques" + '/'  + ' {}'.format(TODAY) + " {}".format(portefeuille[-16:-5]) + ' Statistiques_Portefeuilles' +' {}'.format(window_backtest) + ' {}'.format(window_lenght)  +' {}'.format(MAR) + '.xlsx')
            Noms_Drawdowns.append(Liste_for_database_mngt[k] + "/Drawdowns csv" + '/' +' {}'.format(TODAY) + " Drawdown du Portefeuille" + " {}".format(portefeuille[-16:-5]) + ' {}'.format(window_backtest) +' MAR' +' {}'.format(MAR) +'.csv')
            Noms_Equity_Curves.append(Liste_for_database_mngt[k] + "/Equity Curves" + '/'  +' {}'.format(TODAY) + " {}".format(portefeuille[-16:-5]) +  " equity Curve"+ ' {}'.format(window_backtest) +' MAR' + ' {}'.format(MAR) + ".csv")
            
            k = k + 1    
            
        Table_de_correspondance = Noms_Statistiques
        Table_de_correspondance = pd.DataFrame(Table_de_correspondance)
        Table_de_correspondance["Drawdowns"] = Noms_Drawdowns
        Table_de_correspondance["Equity Curves"] = Noms_Equity_Curves


    return  Table_de_correspondance

# Fonction de trie des résultats selon un critère:
# Input : Le filtre : string, le nombre de data à afficher : int,La liste des datafram à lire DataFrame
# Output : Liste de DataFrame trié : DataFrame_List 
def get_sorted(thing,lenght,dataframe):
    
    
    liste_to_append = [] 
    value_list = []
    dataset = pd.concat(dataframe)
    
    for value in dataset.loc[0].values:
    
        df = pd.read_excel(value)

        liste_to_append.append(df)
        value_list.append(value)
        
    output = pd.concat(liste_to_append)
    output["Name"] = value_list
    output.drop(columns = ["Unnamed: 0"],inplace = True)
    output.set_index("Name",inplace =True)
        
    if thing == "Total Return":
        output.sort_values(by = output.columns[0],ascending = False,inplace =True)

    if thing == "Mean Return":
        output.sort_values(by = output.columns[1],ascending = False,inplace =True)

    if thing == "Max Drawdown":
        output.sort_values(by = output.columns[3],ascending = False,inplace =True)

    if thing == "Sharp Ratio":
        output.sort_values(by = output.columns[5],ascending = False,inplace =True)

    if thing == "Time To Recovery":
        output.sort_values(by = output.columns[6],ascending = True,inplace =True)
            
    if thing == "Calmar Ratio":
        output.sort_values(by = output.columns[4],ascending = True,inplace =True)    
            
    return output[:lenght]

# Table de correspondance entre les tableaux contenant les statistiques et les Valeurs à dessiner :
# Input : Liste de Noms de DataFrame Statistique,
def get_dataframes_names(sorted_statistiques_dataframe,Final_dataframe,statistique):
    
    index_wanted = []
    liste = [] 
    
    for k in range (len(sorted_statistiques_dataframe)):
        index_wanted.append(Final_dataframe.index[Final_dataframe[0] == sorted_statistiques_dataframe.index[k]][0])
    
    for index in index_wanted:
        liste.append(Final_dataframe.loc[index][statistique])
        
    return liste

# Fonction pour lire les fichiers dans un dossier:
# Input : String
def get_files_in_folder(path):
    
    Liste_files = []
    
    for files in walk (path):
        Liste_files.extend(files)
        
    Liste_files = Liste_files[2]
    
    return Liste_files

# Fonction d'obtention des multiplots:
# Input : String : noms du dossier contenant les fichiers à dessiner
# Output : Multiple Plots
def get_multiple_plotting(statistique_to_filter,data_to_plot,dataframe_list,final_dataframe,lenght):
    super_liste = []
    for k in range (len(portefeuille_name_list)):
        
        liste_ = []
        
        for value in dataframe_list:
        
            liste_.append(value.loc[k])
            
        path =  portefeuille_name_list[k][:-5] + "/" +  data_to_plot + "/"
        
        sorted_statistiques_dataframe = get_sorted(statistique_to_filter,lenght,liste_)
        
        if data_to_plot == "Drawdowns csv":
            
            value_for_files_names_list = "Drawdowns"
            
        if data_to_plot == "Equity Curves":
            
            value_for_files_names_list = "Equity Curves"    
            
        files_names_list = get_dataframes_names(sorted_statistiques_dataframe,final_dataframe,value_for_files_names_list)
        
        liste = []
        
        for value in files_names_list:

            df = pd.read_csv(value)
            df.set_index(df.columns[0][-10:],inplace = True)
            df.index = pd.to_datetime(df.index)
            df.columns = [value[-24:]]
            liste.append(df)

        liste = pd.concat(liste,axis = 1)
        liste.plot(colormap='gist_yarg',figsize = (20,10))
        plt.title(portefeuille_name_list[k][:-5] + " " + data_to_plot + " trié selon "+ '{}'.format(statistique_to_filter))
        
        if data_to_plot == "Equity Curves":
            
            plt.savefig(portefeuille_name_list[k][:-5] + "/Multiplots_Equity_Curves/" + " {}".format(TODAY) + portefeuille_name_list[k][:-5] + " " + data_to_plot + " filtre selon "+ '{}'.format(statistique_to_filter) + ".png")
          
        if data_to_plot == "Drawdowns csv":
            
            plt.savefig(portefeuille_name_list[k][:-5] + "/Multiplots_Drawdowns/" + " {}".format(TODAY) + portefeuille_name_list[k][:-5] + " " + data_to_plot + " filtre selon "+ '{}'.format(statistique_to_filter) + ".png")
        
        plt.legend()
        super_liste.append(liste) 

    return super_liste 

# Spécifier L'ensemble des paramêtres que cette fonction peut prendre et construire un pipeline qui plot tous les résultats :
Input_list_filters = ["Total Return","Mean Return","Max Drawdown","Sharp Ratio","Time To Recovery","Calmar Ratio"]
Input_list_thing_to_plot = ["Equity Curves","Drawdowns csv"]

def get_all_the_multi_plottings(Input_list_filters,Input_list_thing_to_plot,number):
    
    liste = []
    
    for filtre in Input_list_filters:
        for thing in Input_list_thing_to_plot:
            liste.append(get_multiple_plotting(filtre,thing,dataframe_list_decile,Final_dataframe,number))
            
    return liste    

if Choix_optimisation==1:
    Mar_list = (-0.0150,-0.0125,-0.01,-0.0075,-0.0050,-0.0025,0.0025,0.005,0.0075,0.01,0.0125,0.0150)
    window_backtest_list = (63,93)                                
    import warnings
    warnings.filterwarnings("ignore")
    dataframe_list_decile = []                                    
    for value in window_backtest_list:
        for value_1 in Mar_list:
            dataframe_list_decile.append(optimize(value,value_1))
    Final_dataframe = pd.concat(dataframe_list_decile)
    Final_dataframe.reset_index(inplace = True)
    Final_dataframe.drop(columns = ['index'],inplace = True)
    liste_dataframe_ = get_all_the_multi_plottings(Input_list_filters,Input_list_thing_to_plot,8)
    
if Choix_optimisation==2:
    print("Veuillez Sélectionner le MAR que vous souhaitez : 1.5% == 0.015")
    Mar_value = input()
    Mar_value = float(Mar_value)
    window_backtest_list = (63,93)
    import warnings
    warnings.filterwarnings("ignore")
    for value in window_backtest_list:
          optimize(value,Mar_value)
                                         
if Choix_optimisation==3:
    import warnings
    warnings.filterwarnings("ignore")
    dataframe_list_decile = []
    dataframe_list_decile.append(optimize(0,0))

Veuillez Selectionner l'année de départ pour la fenêtre d'optimisation:


 ert


ValueError: invalid literal for int() with base 10: 'ert'

In [ ]:
print("Veuillez Sélectionner Les paramêtres que vous souhaitez afin d'obtenir une attribution de performance : ")
print("input 1 :window_backtest_1")
window_backtest_1 = input()
window_backtest_1 = int(window_backtest_1)

print("input 2 :window_backtest_2")
window_backtest_2 = input()
window_backtest_2 = int(window_backtest_2)

print("input 3 :window_lenght_1")
window_lenght_1 = input()
window_lenght_1 = int(window_lenght_1)

print("input 4 :window_lenght_2")
window_lenght_2 = input()
window_lenght_2 = int(window_lenght_2)

print("input 5 : mar_value")
mar_value = input()
mar_value = float(mar_value)
#for k in range(len(Liste_for_database_mngt)):
 #   os.mkdir(Liste_for_database_mngt[k] + "/Performances attribution")
    
sub_directory_database_list = ["/Performances attribution"]
sub_directory_weights = ["/Weight matrices"]

for k in range (len(directory_list)):
    directory_path_attributions = Liste_for_database_mngt[k] + sub_directory_database_list[0]

# Fonction de calcul des attributions de performances : 
# Input : int,int,int,int,float
# Elle renvoie 3 graphes, l'un pour le premier paramêtre le second pour le second et le dernier la diiférence des deux.
import datetime 

def get_attributions(window_backtest_1,window_backtest_2,window_lenght_1,window_lenght_2,mar_value):
    
            
    # Backtest_starting_date :     
    backtest_starting_date = datetime.datetime(année_début, mois_début, jour_début)
    backtest_starting_date = backtest_starting_date + BDay(1)
    backtest_starting_date = datetime.datetime(int(str(backtest_starting_date)[:4]), int(str(backtest_starting_date)[5:7]), int(str(backtest_starting_date)[8:10]))
        
    # Backtest_ending_date : 
    backtest_ending_date = datetime.datetime(année_fin,mois_fin,jour_fin)
    backtest_ending_date = backtest_ending_date + BDay(1)
    backtest_ending_date = datetime.datetime(int(str(backtest_ending_date)[:4]), int(str(backtest_ending_date)[5:7]), int(str(backtest_ending_date)[8:10]))
    
    for k,portefeuille in zip (range (len(directory_list)),directory_list):
        
        directory_path_attributions = Liste_for_database_mngt[k] + sub_directory_database_list[0]
        Portefeuille = pd.read_excel(directory_list[k])

        Portefeuille_list1 = prepare_portefeuille(Portefeuille,window_backtest_1,window_lenght_1,backtest_starting_date,backtest_ending_date)
        Portefeuille_list2 = prepare_portefeuille(Portefeuille,window_backtest_2,window_lenght_2,backtest_starting_date,backtest_ending_date)
        
        Weight_Matrice_1 = pd.read_excel(Liste_for_database_mngt[k] + "/Weight matrices" + '/'  + "{} ".format(TODAY) + "{} ".format(portefeuille[-16:-5]) + get_weight_name("Weight_matrice",window_backtest_1,mar_value))
        Weight_Matrice_2 = pd.read_excel(Liste_for_database_mngt[k] + "/Weight matrices" + '/'  + "{} ".format(TODAY) + "{} ".format(portefeuille[-16:-5]) + get_weight_name("Weight_matrice",window_backtest_2,mar_value))
        
        Weight_Matrice_1.set_index('Unnamed: 0',inplace =True)
        Weight_Matrice_2.set_index('Unnamed: 0',inplace =True)

        Realized_return_1 = get_realized_returns(Portefeuille_list1,Weight_Matrice_1)
        Realized_return_2 = get_realized_returns(Portefeuille_list2,Weight_Matrice_2)
        
        index = len(Realized_return_2) - 5
        
        plt.figure(figsize = (20,10))
        Realized_return_1.plot(colormap='PuRd',figsize = (20,10))
        plt.title('Attribution de performance'+ ' {}'.format(portefeuille[-16:-5]) + ' {}'.format(window_backtest_1))
        plt.savefig(directory_path_attributions +  "/" + 'Attribution de performance'+ ' {}'.format(portefeuille[-16:-5]) + ' {}'.format(window_backtest_1) + ".png")
        
        plt.figure(figsize = (20,10))
        Realized_return_2.plot(colormap='PuRd',figsize = (20,10))
        plt.title('Attribution de performance' + ' {}'.format(portefeuille[-16:-5]) + ' {}'.format(window_backtest_2))
        plt.savefig(directory_path_attributions +  "/" + 'Attribution de performance' + ' {}'.format(portefeuille[-16:-5]) + ' {}'.format(window_backtest_2) + ".png")
        
        plt.figure(figsize = (20,10))
        last_df = Realized_return_1[-index:] - Realized_return_2[-index:]
        last_df.plot(colormap='PuRd',figsize = (20,10))
        plt.title('Attribution de performance' + ' {}'.format(portefeuille[-16:-5]) + ' {}'.format(window_backtest_1) + " - " +' {}'.format(window_backtest_2))
        plt.savefig(directory_path_attributions + "/" + 'Attribution de performance' + ' {}'.format(portefeuille[-16:-5]) + ' {}'.format(window_backtest_1) + " - " +' {}'.format(window_backtest_2) + ".png")
        
        
get_attributions(window_backtest_1,window_backtest_2,window_lenght_1,window_lenght_2,mar_value)

Veuillez Sélectionner Les paramêtres que vous souhaitez afin d'obtenir une attribution de performance : 
input 1 :window_backtest_1


 2021


input 2 :window_backtest_2


 2


input 3 :window_lenght_1


 21


input 4 :window_lenght_2


 2022


input 5 : mar_value


 11
